In [1]:
import numpy as np

from scipy.sparse import csr_matrix, lil_matrix, diags
def get_atom_type(atom):
    """
    Return atom type and position index, where atom type includes the site index from
    parentheses appended to the main atom type, e.g., "C2(3)" -> "C3", 2.

    :param atom: str - atom description
    :return: tuple - (atom type with site, position index)
    """
    # The pattern matches an initial letter sequence for the atom type,
    # followed by a number for the position, and numbers in parentheses for site index to be appended.
    pattern = r"([A-Za-z]+)(\d+)\((\d+)\)"
    match = re.match(pattern, atom)
    
    if match:
        # If the pattern matches, append the site index to the atom type
        atom_type = match.group(1) + match.group(3)
        position_index = int(match.group(2))
        return atom_type, position_index
    else:
        # Fallback for handling cases without parentheses
        i = 1
        while i <= len(atom) and atom[-i].isdigit():
            i += 1
        i -= 1
        if i == 0:
            raise ValueError(f"Invalid atom format: {atom}")
        return atom[:-i], int(atom[-i:])
 
    
def generate_connectivity_matrix(connectivity):
    r"""
    Generate connectivity matrix.

    Returns
    -------
    tuple
        (dictionary, np.ndarray)
    """
    max_site = 0
    atoms_sites_lst = []

    # check if self.connectivity is a matrix
    # if so, put assign it to self.connectivity_matrix
    # and set the atom_types to None
    if isinstance(self.connectivity, np.ndarray):
        self.connectivity_matrix = csr_matrix(self.connectivity)
        self.atom_types = None
        self.n_sites = self.connectivity_matrix.shape[0]

        return None, self.connectivity_matrix

    for atom1, atom2, bond in self.connectivity:
        atom1_name, site1 = get_atom_type(atom1)
        atom2_name, site2 = get_atom_type(atom2)
        for pair in [(atom1_name, site1), (atom2_name, site2)]:
            if pair not in atoms_sites_lst:
                atoms_sites_lst.append(pair)
        if max_site < max(site1, site2):  # finding the max index of site
            max_site = max(site1, site2)
    self.n_sites = len(atoms_sites_lst)

    if self.atom_types is None:
        atom_types = [None for i in range(max_site + 1)]
        for atom, site in atoms_sites_lst:
            atom_types[site] = atom
        self.atom_types = atom_types
    connectivity_mtrx = np.zeros((max_site, max_site))

    for atom1, atom2, bond in self.connectivity:
        atom1_name, site1 = get_atom_type(atom1)
        atom2_name, site2 = get_atom_type(atom2)
        connectivity_mtrx[site1 - 1, site2 - 1] = bond
        # numbering of sites starts from 1

    connectivity_mtrx = np.maximum(connectivity_mtrx, connectivity_mtrx.T)
    self.connectivity_matrix = csr_matrix(connectivity_mtrx)
    return atoms_sites_lst, self.connectivity_matrix

generate_connectivity_matrix([('C1', 'C2', 1), ('C2', 'C1', 1)])


AttributeError: 'list' object has no attribute 'connectivity'